In [1]:
import os
from dotenv import load_dotenv
from paths import PARENT_DIR, RAW_DATA_DIR, BRONZE_DATA_DIR, SILVER_DATA_DIR, GOLD_DATA_DIR
from data import transform_save_data_into_ts_data, validate_and_save_data
from datetime import datetime
import pandas as pd
from pathlib import Path
import hopsworks

2024-08-03 19:41:05,139 - INFO - Folder "data" ensured at "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data"
2024-08-03 19:41:05,139 - INFO - Folder "raw" ensured at "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data/raw"
2024-08-03 19:41:05,139 - INFO - Folder "processed" ensured at "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data/bronze"
2024-08-03 19:41:05,140 - INFO - Folder "transformed" ensured at "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data/silver"
2024-08-03 19:41:05,140 - INFO - Folder "transformed" ensured at "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data/gold"


In [2]:
# Path to the root directory of the project in hopsworks
HOPSWORKS_PROJECT_NAME = 'demand_predictor_borja'

# Load the environment variables
load_dotenv(PARENT_DIR / '.env')    

HOPSWORKS_API_KEY = os.environ.get('HOPSWORKS_API_KEY')

In [3]:

# Define the file paths
file_2023 = Path(f"{SILVER_DATA_DIR}/2023/ts_data_2023.parquet")
file_2024 = Path(f"{SILVER_DATA_DIR}/2024/ts_data_2024.parquet")

# Read the parquet files
df_2023 = pd.read_parquet(file_2023)
df_2024 = pd.read_parquet(file_2024)

# Concatenate the DataFrames
combined_df = pd.concat([df_2023, df_2024], ignore_index=True)

# Define the output file path
output_file = Path(f"{SILVER_DATA_DIR}/2023-2024/ts_data_2023-2024.parquet")

# Save the combined DataFrame to a new parquet file
combined_df.to_parquet(output_file)

print(f"Combined parquet file saved to {output_file}")
print(f"Number of rides: {combined_df.shape[0]}")

Combined parquet file saved to /Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data/silver/2023-2024/ts_data_2023-2024.parquet
Number of rides: 3202920


In [11]:
combined_df.head()

,pickup_time,pickup_location_id,ride_count,pickup_ts
0,2023-01-01,4,18.0,1672531200
1,2023-01-01,7,3.0,1672531200
2,2023-01-01,12,1.0,1672531200
3,2023-01-01,13,14.0,1672531200
4,2023-01-01,24,20.0,1672531200


In [10]:
# Convert 'pickup_time' to Unix timestamp (BIGINT) so that it can be stored
combined_df['pickup_ts'] = pd.to_datetime(combined_df['pickup_time']).astype(int) // 10**9

In [12]:
project = hopsworks.login(project = HOPSWORKS_PROJECT_NAME,
                          api_key_value = HOPSWORKS_API_KEY
                          )

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/933012


In [13]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [14]:
FEATURE_GROUP_NAME = 'hourly_features_2023_2024'    
FEATURE_GROUP_VERSION = 1

In [17]:
feature_group = feature_store.get_or_create_feature_group(
  name=FEATURE_GROUP_NAME , 
  version=FEATURE_GROUP_VERSION, 
  description='time series hourly features for 2023-2024 cab rides',
  primary_key=['pickup_ts', 'pickup_location_id'], 
  event_time='pickup_ts',
  online_enabled=True,
)

In [18]:
feature_group.save(combined_df, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/933012/fs/926787/fg/1067344


Uploading Dataframe: 0.00% |          | Rows 0/3202920 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: hourly_features_2023_2024_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/933012/jobs/named/hourly_features_2023_2024_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x3115ad3d0>, None)